In [1]:
import tensorflow as tf

In [4]:
import numpy as np


In [6]:
# Using Kaiming He Initialization for kernel nitialization
def init_kaiming_he(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [7]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_kaiming_he, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

In [8]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)
      
  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h Creating model using subclassing API. 
# Same can be achieved using Functional API or Sequential API
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_kaiming_he, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

SyntaxError: ignored

In [9]:
class CIFAR10AutoGrad(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    self.maxpool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*3, self.maxpool, res = True)
    self.blk2 = ResBlk(c*6, self.maxpool)
    self.blk3 = ResBlk(c*9, self.maxpool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_kaiming_he, use_bias=False)
    self.weight = weight
# Building the model
  def call(self, x, y):
    h = self.pool(self.maxpool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y) 
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [11]:
import time

In [12]:
t = time.time()

def fit():
  # Create batches for test data
  test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug,num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
  #summary_writer = tf.summary.create_file_writer('./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")))
  train_summary_writer = tf.summary.create_file_writer('./logs/train')
  test_summary_writer = tf.summary.create_file_writer('./logs/test')    

  for epoch in range(EPOCHS):
    train_loss = test_loss = train_acc = test_acc = 0.0
    with train_summary_writer.as_default():
      tf.summary.trace_on(graph=True, profiler=True) # To start capturing profiler in Tensorboard
      tf.keras.backend.set_learning_phase(1)
      for (x, y) in tqdm(train_set):# Iterate over the batches of train dataset objects .
        with tf.GradientTape() as tape:
          #logits,actual  value for this minibatch
          loss, correct = model(x, y)
          var = model.trainable_variables
          grads = tape.gradient(loss, var)
          opt.apply_gradients(zip(grads, var))
          global_step.assign_add(1)
        # Add extra losses created during this forward pass
        train_loss += loss.numpy()
        train_acc += correct.numpy()

      tf.summary.scalar('train loss', train_loss/len_train, step=epoch)
      tf.summary.scalar('train acc', (train_acc/len_train)*100, step=epoch)
      tf.summary.trace_export(name="Train", step=epoch,profiler_outdir='./logs/train/trace') # Close Profiling when we do export
      tf.keras.backend.set_learning_phase(0)
      
    with test_summary_writer.as_default(): 
      tf.summary.trace_on(graph=True, profiler=True)
      for (x, y) in test_set:#  Iterate over the batches of train dataset objects .

        loss, correct = model(x, y)
        test_loss += loss.numpy()
        test_acc += correct.numpy()         
      tf.summary.scalar('test loss', test_loss/len_test, step=epoch)
      tf.summary.scalar('test acc', (test_acc/len_test)*100, step=epoch) 
      tf.summary.trace_export(name="Test", step=epoch,profiler_outdir='./logs/test/trace')  

      template = 'Epoch {}, lr:{:.3f},Train Loss: {:.3f},Train Accuracy: {:.3f}, Test Loss: {:.3f}, Test Accuracy: {:.3f},Time Taken: {:.2f}'
      print (template.format(epoch+1,lr_schedule(epoch+1),train_loss/len_train,(train_acc/len_train)*100,test_loss/len_test,(test_acc/len_test)*100,time.time() - t))

In [13]:
tf.summary.create_file_writer('./logs/train') 


In [14]:
      print (template.format(epoch+1,lr_schedule(epoch+1),train_loss/len_train,(train_acc/len_train)*100,test_loss/len_test,(test_acc/len_test)*100,time.time() - t))

NameError: ignored